<a href="https://colab.research.google.com/github/PujanPandey07/Movie-Recommender-system-using-NLP-and-ML/blob/main/preprocessingdm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving tmdb_5000_movies.csv to tmdb_5000_movies.csv


## loading the datset files


In [ ]:
import pandas as pd
df1 = pd.read_csv("tmdb_5000_movies.csv")
df2=pd.read_csv("tmdb_5000_credits.csv")


## lets see the features of our datset

In [ ]:
df1.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [ ]:
df2.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


##Extracting reviews for each movie from tmdb website

In [ ]:
import requests
import time
import pandas as pd
import re
from google.colab import files



TMDB_API_KEY = "dedddb77b7d3572bb3576a7c969e4580"

def clean_text(text):
    # Keep only letters, numbers, spaces, and periods
    return re.sub(r'[^a-zA-Z0-9\s\.]', '', text)

def get_all_reviews(movie_id):
    reviews = []
    page = 1
    while True:
        url = f"https://api.themoviedb.org/3/movie/{movie_id}/reviews?api_key={TMDB_API_KEY}&page={page}"
        try:
            res = requests.get(url)
            data = res.json()
            results = data.get('results', [])
            if not results:
                break  # No more reviews

            for review in results:
                content = review.get('content', '')
                cleaned = clean_text(content)
                reviews.append(cleaned)

            if page >= data.get('total_pages', 1):
                break  # Last page
            page += 1
            time.sleep(0.3)  # Rate limit protection
        except Exception as e:
            print(f"Error fetching review for ID {movie_id} on page {page}: {e}")
            break
    return reviews if reviews else "No reviews available"

# === Loop through all movies and fetch cleaned reviews
all_reviews = []

for idx, row in df1.iterrows():
    movie_id = row['id']
    reviews_text = get_all_reviews(movie_id)
    all_reviews.append(reviews_text)

    if idx % 50 == 0:
        print(f"Processed {idx+1}/{len(df1)} movies...")

# Add new column
df1['reviews'] = all_reviews


Processed 1/4803 movies...
Processed 51/4803 movies...
Processed 101/4803 movies...
Processed 151/4803 movies...
Processed 201/4803 movies...
Processed 251/4803 movies...
Processed 301/4803 movies...
Processed 351/4803 movies...
Processed 401/4803 movies...
Processed 451/4803 movies...
Processed 501/4803 movies...
Processed 551/4803 movies...
Processed 601/4803 movies...
Processed 651/4803 movies...
Processed 701/4803 movies...
Processed 751/4803 movies...
Processed 801/4803 movies...
Processed 851/4803 movies...
Processed 901/4803 movies...
Processed 951/4803 movies...
Processed 1001/4803 movies...
Processed 1051/4803 movies...
Processed 1101/4803 movies...
Processed 1151/4803 movies...
Processed 1201/4803 movies...
Processed 1251/4803 movies...
Processed 1301/4803 movies...
Processed 1351/4803 movies...
Processed 1401/4803 movies...
Processed 1451/4803 movies...
Processed 1501/4803 movies...
Processed 1551/4803 movies...
Processed 1601/4803 movies...
Processed 1651/4803 movies...
Pro

## merging two datset files

In [ ]:

movies_df = df1.merge(df2, left_on='id', right_on='movie_id')

## extract required columns from the merged dataset

In [ ]:
movies_df=movies_df[['movie_id','genres','overview','title_x','cast','crew','keywords','reviews']]

In [ ]:
import json

## Extract direcror name


In [ ]:
def get_director(crew_str):
    crew = json.loads(crew_str)
    for member in crew:
        if member['job'] == 'Director':
            return member['name']
    return None

movies_df.loc[:, 'director'] = movies_df['crew'].apply(get_director)


##Extract Main cast

In [ ]:
def get_cast_names(cast_str, top_n=4):
    cast = json.loads(cast_str)
    return [member['name'] for member in cast[:top_n]]

movies_df.loc[:, 'main_cast'] = movies_df['cast'].apply(lambda x: get_cast_names(x, top_n=4))


## creating combined metadata text

In [ ]:
def create_metadata_text(row):
    director = row['director'] or ''
    cast = ' '.join(row['main_cast']) if isinstance(row['main_cast'], list) else ''
    keywords = ' '.join([kw['name'] for kw in json.loads(row['keywords'])]) if row['keywords'] else ''
    return f"{director} {cast} {keywords}"

movies_df.loc[:, 'metadata_text'] = movies_df.apply(create_metadata_text, axis=1)


In [ ]:
print(movies_df.columns.tolist())


['movie_id', 'genres', 'overview', 'title_x', 'cast', 'crew', 'keywords', 'reviews', 'director', 'main_cast', 'metadata_text']


## saving new datset

In [ ]:
movies_df.to_csv("movies_df_processed.csv", index=False)


In [ ]:
from google.colab import files
files.download('movies_df_processed.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>